# control_iSarcasm
This notebook trains our control model--a model that is only fine-tuned on our target task, iSarcasm.

## Platform Check
Ensure we're on an ARM environment. 

In [1]:
import platform

if platform.platform() == 'macOS-13.0-arm64-i386-64bit':
    print(f"We're Armed: {platform.platform()}")
else:
    print(f"WARNING! NOT ARMED: {platform.platform()}")

We're Armed: macOS-13.0-arm64-i386-64bit


## Imports & Settings

First, update working directory to parent so that we may use our custom functions

In [2]:
import os
 
os.chdir('..')
os.getcwd( )

'/Users/jarradjinx/Library/Mobile Documents/com~apple~CloudDocs/EDU_leeds/LD_research/LD_ITESD'

In [3]:
import params
from utils import *
from trainer import *

import numpy as np
import pandas as pd

from tqdm import tqdm
from tqdm import trange

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

from transformers import RobertaTokenizer, RobertaForSequenceClassification
# from transformers import BertTokenizer, BertForSequenceClassification

# suppress model warning
from transformers import logging
logging.set_verbosity_error()

# set logging level
import logging
logging.basicConfig(level='INFO')

# set general seeds
set_seeds(1)

# set dataloader generator seed
g = torch.Generator()
g.manual_seed(1)

## Load Data

### iSarcasm

In [4]:
dataset_path = 'data/target_semEval2022_en/iSarcasmEval-main/train/train.en.prepped-oversampled.csv'
df = pd.read_csv(dataset_path)
df = df.rename(columns={'tweet': 'text'})
df = df.rename(columns={'sarcastic': 'label'})

df.head()

,text,label
0,@ThePartridgePod Defrost the freezer with the ...,1
1,really honoured to have shared a coach today w...,1
2,I can't believe today is the last day we can b...,1
3,"thank you kind person that stole my ATM card, ...",1
4,I love leaving the doctors office in tears. I ...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5200 non-null   object
 1   label   5200 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 81.4+ KB


In [6]:
df['label'].value_counts()

1    2600
0    2600
Name: label, dtype: int64

#### Sampling
The Sampling function below allows us to adjust the size of our dataset for quick testing.

In [ ]:
sample_amounts = {0: 300, 1:300}

df = (
    df.groupby('label').apply(lambda g: g.sample(
        # lookup number of samples to take
        n=sample_amounts[g.name],
        # enable replacement if len is less than number of samples expected
        replace=len(g) < sample_amounts[g.name]  
    ))
)

In [ ]:
df['label'].value_counts()

### Target Text & Labels

In [7]:
df['label'].value_counts()

1    2600
0    2600
Name: label, dtype: int64

In [8]:
text = df.text.values
labels = df.label.values

## Preprocess

In [9]:
token_id = []
attention_masks = []

for sample in text:
  encoding_dict = preprocessing(sample, params.tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

## Data Split
We split the dataset into train (80%) and validation (20%) sets, and wrap them around a torch.utils.data.DataLoader object.

In [10]:
val_ratio = 0.2

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels,
    random_state=1)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
        )

## Train

Download transformers.RobertaForSequenceClassificatio, which is a RoBERTa model with a linear layer for sentence classification (or regression) on top of the pooled output:

In [11]:
# Load the RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                         num_labels = params.num_labels,
                                                         output_attentions = False,
                                                         output_hidden_states = False,
                                                         )

from torchinfo import summary
summary(model, input_size=(1, 512), dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                       Output Shape              Param #
RobertaForSequenceClassification                             [1, 2]                    --
├─RobertaModel: 1-1                                          [1, 512, 768]             --
│    └─RobertaEmbeddings: 2-1                                [1, 512, 768]             --
│    │    └─Embedding: 3-1                                   [1, 512, 768]             38,603,520
│    │    └─Embedding: 3-2                                   [1, 512, 768]             768
│    │    └─Embedding: 3-3                                   [1, 512, 768]             394,752
│    │    └─LayerNorm: 3-4                                   [1, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 512, 768]             --
│    └─RobertaEncoder: 2-2                                   [1, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --               

Set model to device, initialize trainer

In [12]:
model.to(params.device)
print(f"Trained Dataset: {dataset_path}")
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), lr=params.learning_rate) #roberta

trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq)

Trained Dataset: data/target_semEval2022_en/iSarcasmEval-main/train/train.en.prepped-oversampled.csv
Device: mps


Fit the model to our training data.

In [13]:
trainer.fit()

Epoch 1:   0%|          | 0/260 [00:00<?, ?batch/s]/Users/jarradjinx/opt/anaconda3/envs/itesd_env/lib/python3.9/site-packages/transformers/models/roberta/modeling_roberta.py:1570: UserWarning: The operator 'aten::cumsum.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  incremental_indices = (torch.cumsum(mask, dim=1).type_as(mask) + past_key_values_length) * mask
	 Validation 64: 100%|██████████| 65/65 [00:25<00:00,  2.51batch/s]


	 - Train loss: 0.687505
	 - Validation Loss: 0.633956
	 - Validation Accuracy: 0.683654
	 - Validation F1: 0.674069
	 - Validation Recall: 0.707433
	 - Validation Precision: 0.666377
	 * Model @ epoch 1 saved to model_saves/final_update_test/E01_A0.68_F0.67
	 * Model checkpoint saved to model_saves/final_update_test/E01_A0.68_F0.67/checkpoint.pt


	 Validation 64: 100%|██████████| 65/65 [00:26<00:00,  2.47batch/s]


	 - Train loss: 0.514968
	 - Validation Loss: 0.372114
	 - Validation Accuracy: 0.826923
	 - Validation F1: 0.820621
	 - Validation Recall: 0.849182
	 - Validation Precision: 0.812026
	 * Model @ epoch 2 saved to model_saves/final_update_test/E02_A0.83_F0.82
	 * Model checkpoint saved to model_saves/final_update_test/E02_A0.83_F0.82/checkpoint.pt


	 Validation 64: 100%|██████████| 65/65 [00:25<00:00,  2.52batch/s]


	 - Train loss: 0.237459
	 - Validation Loss: 0.311567
	 - Validation Accuracy: 0.883654
	 - Validation F1: 0.882541
	 - Validation Recall: 0.925095
	 - Validation Precision: 0.854121
	 * Model @ epoch 3 saved to model_saves/final_update_test/E03_A0.88_F0.88
	 * Model checkpoint saved to model_saves/final_update_test/E03_A0.88_F0.88/checkpoint.pt


Epoch 4:   7%|▋         | 18/260 [00:26<06:01,  1.50s/batch]


KeyboardInterrupt: 

In [ ]:
# Load the RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                         num_labels = params.num_labels,
                                                         output_attentions = False,
                                                         output_hidden_states = False,
                                                         )

model.to(params.device)
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), lr=params.learning_rate) #roberta

checkpoint_load = "model_saves/new_func_test/E02_A0.85_F0.84/checkpoint.pt"
trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  phone_number=params.phone_number,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq, 
                  checkpoint_load=checkpoint_load)

In [ ]:
trainer.fit()

## Load & Predict

### Full Test

In [ ]:
from transformers import TextClassificationPipeline
from transformers import AutoModelForSequenceClassification

PATH = 'model_saves/bert_sarc_long_test/E04_A0.92_F0.91/'
model = AutoModelForSequenceClassification.from_pretrained(PATH, local_files_only=True)
tokenizer = RobertaTokenizer.from_pretrained(PATH, local_files_only=True)

# define pipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=2)

In [ ]:
df = pd.read_csv('data/target_semEval2022_en/iSarcasmEval-main/test/task_A_En_test.csv')
df = df.rename(columns={'tweet': 'text'})
df = df.rename(columns={'sarcastic': 'label'})

df.head()

In [ ]:
test_input = df['text'].to_list()

test_output = []

# run tests and append to output
with tqdm(test_input, unit="test") as prog:
    for step, test in enumerate(prog):
        prog.set_description(f"Test {step}")
        test_output.append(pipe(test)[0])

In [ ]:
# parse predictions to new list
predictions = []

for i in test_output:
    predictions.append(i[0]['label'])
    
print(len(predictions))

In [ ]:
df['preds'] = predictions
df["preds"] = df["preds"].str.replace("LABEL_","")
df['preds'] = df["preds"].astype(int)
df.tail()

In [ ]:
df.info()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# epoch 3
acc = accuracy_score(df['label'], df['preds'])
f1 = f1_score(df['label'], df['preds'])

print(acc)
print(f1)

In [ ]:
print(1e-05)
print(type(1e-05))